# Introduction

In this guide, we will walk you through building a powerful semantic search engine using [Couchbase](https://www.couchbase.com) as the backend database and [CrewAI](https://github.com/crewAIInc/crewAI) for agent-based RAG operations. CrewAI allows us to create specialized agents that can work together to handle different aspects of the RAG workflow, from document retrieval to response generation. This tutorial uses Couchbase's **Global Secondary Index (GSI)** vector search capabilities, which offer high-performance vector search optimized for large-scale applications. This tutorial is designed to be beginner-friendly, with clear, step-by-step instructions that will equip you with the knowledge to create a fully functional semantic search system from scratch. Alternatively if you want to perform semantic search using the FTS index, please take a look at [this.](../fts/RAG_with_Couchbase_and_CrewAI.ipynb)

How to run this tutorial
----------------------
This tutorial is available as a Jupyter Notebook (.ipynb file) that you can run 
interactively. You can access the original notebook here.

You can either:
- Download the notebook file and run it on [Google Colab](https://colab.research.google.com)
- Run it on your system by setting up the Python environment

Before you start
---------------

1. Create and Deploy Your Free Tier Operational cluster on [Capella](https://cloud.couchbase.com/sign-up)
   - To get started with [Couchbase Capella](https://cloud.couchbase.com), create an account and use it to deploy 
     a forever free tier operational cluster
   - This account provides you with an environment where you can explore and learn 
     about Capella with no time constraint
   - To learn more, please follow the [Getting Started Guide](https://docs.couchbase.com/cloud/get-started/create-account.html)
   - **Important**: This tutorial requires Couchbase Server **8.0+** for GSI vector search capabilities

2. Couchbase Capella Configuration
   When running Couchbase using Capella, the following prerequisites need to be met:
   - Create the database credentials to access the required bucket (Read and Write) used in the application
   - Allow access to the Cluster from the IP on which the application is running by following the [Network Security documentation](https://docs.couchbase.com/cloud/security/security.html#public-access)

# Pre-requisite Setup

## Installing Necessary Libraries

We'll install the following key libraries:
- `datasets`: For loading and managing our training data
- `langchain-couchbase`: To integrate Couchbase with LangChain for GSI vector storage and caching
- `langchain-openai`: For accessing OpenAI's embedding and chat models
- `crewai`: To create and orchestrate our AI agents for RAG operations
- `python-dotenv`: For securely managing environment variables and API keys

These libraries provide the foundation for building a semantic search engine with GSI vector embeddings, 
database integration, and agent-based RAG capabilities.

In [1]:
%pip install --quiet datasets==4.1.0 langchain-couchbase==0.5.0rc1 langchain-openai==0.3.33 crewai==0.186.1 python-dotenv==1.1.1

Note: you may need to restart the kernel to use updated packages.


## Importing Necessary Libraries

The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading.

In [2]:
import getpass
import json
import logging
import os
import time
from datetime import timedelta
from uuid import uuid4

from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.diagnostics import PingState, ServiceType
from couchbase.exceptions import (InternalServerFailureException,
                                  QueryIndexAlreadyExistsException,
                                  ServiceUnavailableException,
                                  CouchbaseException)
from couchbase.management.buckets import CreateBucketSettings
from couchbase.options import ClusterOptions
from datasets import load_dataset
from dotenv import load_dotenv
from crewai.tools import tool
from langchain_couchbase.vectorstores import CouchbaseQueryVectorStore
from langchain_couchbase.vectorstores import DistanceStrategy, IndexType
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from crewai import Agent, Crew, Process, Task

/Users/viraj.agarwal/Tasks/Task10/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Logging

Logging is configured to track the progress of the script and capture any errors or warnings.

In [4]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Suppress httpx logging
logging.getLogger('httpx').setLevel(logging.CRITICAL)

## Loading Sensitive Information

In this section, we prompt the user to input essential configuration settings needed. These settings include sensitive information like database credentials, and specific configuration names. Instead of hardcoding these details into the script, we request the user to provide them at runtime, ensuring flexibility and security.

The script uses environment variables to store sensitive information, enhancing the overall security and maintainability of your code by avoiding hardcoded values.

In [5]:
# Load environment variables
load_dotenv("./.env")

# Configuration
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or input("Enter your OpenAI API key: ")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set")

CB_HOST = os.getenv('CB_HOST') or input("Enter Couchbase host (default: couchbase://localhost): ") or 'couchbase://localhost'
CB_USERNAME = os.getenv('CB_USERNAME') or input("Enter Couchbase username (default: Administrator): ") or 'Administrator'
CB_PASSWORD = os.getenv('CB_PASSWORD') or getpass.getpass("Enter Couchbase password (default: password): ") or 'password'
CB_BUCKET_NAME = os.getenv('CB_BUCKET_NAME') or input("Enter bucket name (default: vector-search-testing): ") or 'vector-search-testing'
INDEX_NAME = os.getenv('INDEX_NAME') or input("Enter index name (default: vector_search_crew): ") or 'vector_search_crew'
SCOPE_NAME = os.getenv('SCOPE_NAME') or input("Enter scope name (default: shared): ") or 'shared'
COLLECTION_NAME = os.getenv('COLLECTION_NAME') or input("Enter collection name (default: crew): ") or 'crew'

print("Configuration loaded successfully")

Configuration loaded successfully


## Connecting to the Couchbase Cluster

Connecting to a Couchbase cluster is the foundation of our project. Couchbase will serve as our primary data store, handling all the storage and retrieval operations required for our semantic search engine. By establishing this connection, we enable our application to interact with the database, allowing us to perform operations such as storing embeddings, querying data, and managing collections. This connection is the gateway through which all data will flow, so ensuring it's set up correctly is paramount.

In [6]:
# Connect to Couchbase
try:
    auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
    options = ClusterOptions(auth)
    cluster = Cluster(CB_HOST, options)
    cluster.wait_until_ready(timedelta(seconds=5))
    print("Successfully connected to Couchbase")
except Exception as e:
    print(f"Failed to connect to Couchbase: {str(e)}")
    raise

Successfully connected to Couchbase


## Verifying Query Service Availability

In this section, we verify that the Couchbase Query service is available and responding correctly. This is a crucial check because our GSI vector search functionality depends on it. If any issues are detected with the Query service, the function will raise an exception, allowing us to catch and handle problems early before attempting GSI vector operations.

In [7]:
def check_query_service(cluster):
    """Verify query service availability using ping"""
    try:
        # Get ping result
        ping_result = cluster.ping()
        query_available = False
        
        # Check if query service is responding
        for service_type, endpoints in ping_result.endpoints.items():
            if service_type == ServiceType.Query:
                for endpoint in endpoints:
                    if endpoint.state == PingState.OK:
                        query_available = True
                        print(f"Query service is responding at: {endpoint.remote}")
                        break
                break

        if not query_available:
            raise RuntimeError("Query service not found or not responding")
        
        print("Query service check passed successfully")
    except Exception as e:
        print(f"Health check failed: {str(e)}")
        raise
try:
    check_query_service(cluster)
except Exception as e:
    print(f"Failed to check query service: {str(e)}")
    raise

Query service is responding at: [::1]:8093
Query service check passed successfully


## Collections in Couchbase

The setup_collection() function handles creating and configuring the hierarchical data organization in Couchbase:

1. Bucket Creation:
   - Checks if specified bucket exists, creates it if not
   - Sets bucket properties like RAM quota (1024MB) and replication (disabled)
   - Note: If you are using Capella, create a bucket manually called vector-search-testing(or any name you prefer) with the same properties.

2. Scope Management:  
   - Verifies if requested scope exists within bucket
   - Creates new scope if needed (unless it's the default "_default" scope)

3. Collection Setup:
   - Checks for collection existence within scope
   - Creates collection if it doesn't exist
   - Waits 2 seconds for collection to be ready

Additional Tasks:
- Creates primary index on collection for query performance
- Clears any existing documents for clean state
- Implements comprehensive error handling and logging

The function is called twice to set up:
1. Main collection for vector embeddings
2. Cache collection for storing results


In [9]:
def setup_collection(cluster, bucket_name, scope_name, collection_name):
    try:
        # Check if bucket exists, create if it doesn't
        try:
            bucket = cluster.bucket(bucket_name)
            logging.info(f"Bucket '{bucket_name}' exists.")
        except Exception as e:
            logging.info(f"Bucket '{bucket_name}' does not exist. Creating it...")
            bucket_settings = CreateBucketSettings(
                name=bucket_name,
                bucket_type='couchbase',
                ram_quota_mb=1024,
                flush_enabled=True,
                num_replicas=0
            )
            cluster.buckets().create_bucket(bucket_settings)
            time.sleep(2)  # Wait for bucket creation to complete and become available
            bucket = cluster.bucket(bucket_name)
            logging.info(f"Bucket '{bucket_name}' created successfully.")

        bucket_manager = bucket.collections()

        # Check if scope exists, create if it doesn't
        scopes = bucket_manager.get_all_scopes()
        scope_exists = any(scope.name == scope_name for scope in scopes)
        
        if not scope_exists and scope_name != "_default":
            logging.info(f"Scope '{scope_name}' does not exist. Creating it...")
            bucket_manager.create_scope(scope_name)
            logging.info(f"Scope '{scope_name}' created successfully.")

        # Check if collection exists, create if it doesn't
        collections = bucket_manager.get_all_scopes()
        collection_exists = any(
            scope.name == scope_name and collection_name in [col.name for col in scope.collections]
            for scope in collections
        )

        if not collection_exists:
            logging.info(f"Collection '{collection_name}' does not exist. Creating it...")
            bucket_manager.create_collection(scope_name, collection_name)
            logging.info(f"Collection '{collection_name}' created successfully.")
        else:
            logging.info(f"Collection '{collection_name}' already exists. Skipping creation.")

        # Wait for collection to be ready
        collection = bucket.scope(scope_name).collection(collection_name)
        time.sleep(2)  # Give the collection time to be ready for queries

        # Ensure primary index exists
        try:
            cluster.query(f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{bucket_name}`.`{scope_name}`.`{collection_name}`").execute()
            logging.info("Primary index present or created successfully.")
        except Exception as e:
            logging.warning(f"Error creating primary index: {str(e)}")

        # Clear all documents in the collection
        try:
            query = f"DELETE FROM `{bucket_name}`.`{scope_name}`.`{collection_name}`"
            cluster.query(query).execute()
            logging.info("All documents cleared from the collection.")
        except Exception as e:
            logging.warning(f"Error while clearing documents: {str(e)}. The collection might be empty.")

        return collection
    except Exception as e:
        raise RuntimeError(f"Error setting up collection: {str(e)}")
    
setup_collection(cluster, CB_BUCKET_NAME, SCOPE_NAME, COLLECTION_NAME)


2025-09-17 11:30:29 [INFO] Bucket 'vector-search-testing' does not exist. Creating it...


2025-09-17 11:30:31 [INFO] Bucket 'vector-search-testing' created successfully.
2025-09-17 11:30:31 [INFO] Scope 'shared' does not exist. Creating it...
2025-09-17 11:30:31 [INFO] Scope 'shared' created successfully.
2025-09-17 11:30:31 [INFO] Collection 'crew' does not exist. Creating it...
2025-09-17 11:30:31 [INFO] Collection 'crew' created successfully.
2025-09-17 11:30:33 [INFO] Primary index present or created successfully.
2025-09-17 11:30:33 [INFO] All documents cleared from the collection.


# GSI Vector Index Configuration

Semantic search with GSI requires creating a Global Secondary Index optimized for vector operations. Unlike FTS-based vector search, GSI vector indexes offer two distinct types optimized for different use cases:

## GSI Vector Index Types

### Hyperscale Vector Indexes (BHIVE)
- **Best for**: Pure vector searches like content discovery, recommendations, and semantic search
- **Performance**: High performance with low memory footprint, optimized for concurrent operations
- **Scalability**: Designed to scale to billions of vectors
- **Use when**: You primarily perform vector-only queries without complex scalar filtering

### Composite Vector Indexes
- **Best for**: Filtered vector searches that combine vector search with scalar value filtering
- **Performance**: Efficient pre-filtering where scalar attributes reduce the vector comparison scope
- **Use when**: Your queries combine vector similarity with scalar filters that eliminate large portions of data
- **Note**: Scalar filters take precedence over vector similarity

## Understanding Index Configuration

The `index_description` parameter controls how Couchbase optimizes vector storage and search through centroids and quantization:

Format: `'IVF[<centroids>],{PQ|SQ}<settings>'`

Centroids (IVF - Inverted File):
- Controls how the dataset is subdivided for faster searches
- More centroids = faster search, slower training  
- Fewer centroids = slower search, faster training
- If omitted (like IVF,SQ8), Couchbase auto-selects based on dataset size

Quantization Options:
- SQ (Scalar Quantization): SQ4, SQ6, SQ8 (4, 6, or 8 bits per dimension)
- PQ (Product Quantization): PQ<subquantizers>x<bits> (e.g., PQ32x8)
- Higher values = better accuracy, larger index size

Common Examples:
- IVF,SQ8 - Auto centroids, 8-bit scalar quantization (good default)
- IVF1000,SQ6 - 1000 centroids, 6-bit scalar quantization  
- IVF,PQ32x8 - Auto centroids, 32 subquantizers with 8 bits

For detailed configuration options, see the [Quantization & Centroid Settings](https://preview.docs-test.couchbase.com/docs-server-DOC-12565_vector_search_concepts/server/current/vector-index/hyperscale-vector-index.html#algo_settings).

For more information on GSI vector indexes, see [Couchbase GSI Vector Documentation](https://docs.couchbase.com/server/current/vector-index/use-vector-indexes.html).

In [10]:
# GSI Vector Index Configuration
# Unlike FTS indexes, GSI vector indexes are created programmatically through the vector store
# We'll configure the parameters that will be used for index creation

# Vector configuration
VECTOR_DIMENSION = 1536  # OpenAI embedding dimension
DISTANCE_STRATEGY = DistanceStrategy.COSINE  # Cosine similarity
INDEX_TYPE = IndexType.BHIVE  # Using BHIVE for high-performance vector search
INDEX_DESCRIPTION = "IVF,SQ8"  # Auto-selected centroids with 8-bit scalar quantization

print("GSI vector index configuration prepared")

GSI vector index configuration prepared


# OpenAI Configuration

This section initializes two key OpenAI components needed for our RAG system:

1. OpenAI Embeddings:
   - Uses the 'text-embedding-3-small' model
   - Converts text into high-dimensional vector representations (embeddings)
   - These embeddings enable semantic search by capturing the meaning of text
   - Required for vector similarity search in Couchbase

2. ChatOpenAI Language Model:
   - Uses the 'gpt-4o' model
   - Temperature set to 0.2 for balanced creativity and focus
   - Serves as the cognitive engine for CrewAI agents
   - Powers agent reasoning, decision-making, and task execution
   - Enables agents to:
     - Process and understand retrieved context from vector search
     - Generate thoughtful responses based on that context
     - Follow instructions defined in agent roles and goals
     - Collaborate with other agents in the crew
   - The relatively low temperature (0.2) ensures agents produce reliable,
     consistent outputs while maintaining some creative problem-solving ability

Both components require a valid OpenAI API key (OPENAI_API_KEY) for authentication.
In the CrewAI framework, the LLM acts as the "brain" for each agent, allowing them
to interpret tasks, retrieve relevant information via the RAG system, and generate
appropriate outputs based on their specialized roles and expertise.

In [11]:
# Initialize OpenAI components
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    model="text-embedding-3-small"
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4o",
    temperature=0.2
)

print("OpenAI components initialized")

OpenAI components initialized


# Create Couchbase GSI Vector Store

The GSI vector store is where we'll store our embeddings using Couchbase's Global Secondary Index technology. Unlike traditional FTS-based vector search, GSI vector stores are optimized for high-performance vector operations and can scale to billions of vectors. When a user inputs a query, the search engine converts the query into an embedding and uses GSI's optimized vector search capabilities to find the most similar documents.

GSI vector stores offer several advantages:
- **High Performance**: Optimized algorithms for fast vector similarity search
- **Scalability**: Can handle datasets with billions of vectors
- **Memory Efficiency**: Advanced quantization techniques reduce memory usage
- **Concurrent Operations**: Designed to handle multiple simultaneous searches and updates

In [17]:
# Setup GSI vector store with OpenAI embeddings
try:
    vector_store = CouchbaseQueryVectorStore(
        cluster=cluster,
        bucket_name=CB_BUCKET_NAME,
        scope_name=SCOPE_NAME,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        distance_metric=DISTANCE_STRATEGY
    )
    print("GSI Vector store initialized successfully")
    logging.info("GSI Vector store setup completed")
except Exception as e:
    logging.error(f"Failed to initialize GSI vector store: {str(e)}")
    raise RuntimeError(f"GSI Vector store initialization failed: {str(e)}")

2025-09-17 11:31:54 [INFO] GSI Vector store setup completed


GSI Vector store initialized successfully


# Load BBC News Dataset to Couchbase

To build a search engine, we need data to search through. We use the BBC News dataset from RealTimeData, which provides real-world news articles. This dataset contains news articles from BBC covering various topics and time periods. Loading the dataset is a crucial step because it provides the raw material that our search engine will work with. The quality and diversity of the news articles make it an excellent choice for testing and refining our search engine, ensuring it can handle real-world news content effectively.

The BBC News dataset allows us to work with authentic news articles, enabling us to build and test a search engine that can effectively process and retrieve relevant news content. The dataset is loaded using the Hugging Face datasets library, specifically accessing the "RealTimeData/bbc_news_alltime" dataset with the "2024-12" version.

In [18]:
try:
    news_dataset = load_dataset(
        "RealTimeData/bbc_news_alltime", "2024-12", split="train"
    )
    print(f"Loaded the BBC News dataset with {len(news_dataset)} rows")
    logging.info(f"Successfully loaded the BBC News dataset with {len(news_dataset)} rows.")
except Exception as e:
    raise ValueError(f"Error loading the BBC News dataset: {str(e)}")

2025-09-17 11:31:58 [INFO] Successfully loaded the BBC News dataset with 2687 rows.


Loaded the BBC News dataset with 2687 rows


## Cleaning up the Data

We will use the content of the news articles for our RAG system. The dataset contains a few duplicate records. We are removing them to avoid duplicate results in the retrieval stage of our RAG system.

In [19]:
news_articles = news_dataset["content"]
unique_articles = set()
for article in news_articles:
    if article:
        unique_articles.add(article)
unique_news_articles = list(unique_articles)
print(f"We have {len(unique_news_articles)} unique articles in our database.")

We have 1749 unique articles in our database.


## Saving Data to the Vector Store

To efficiently handle the large number of articles, we process them in batches of articles at a time. This batch processing approach helps manage memory usage and provides better control over the ingestion process.

We first filter out any articles that exceed 50,000 characters to avoid potential issues with token limits. Then, using the vector store's add_texts method, we add the filtered articles to our vector database. The batch_size parameter controls how many articles are processed in each iteration.

This approach offers several benefits:
1. Memory Efficiency: Processing in smaller batches prevents memory overload
2. Error Handling: If an error occurs, only the current batch is affected
3. Progress Tracking: Easier to monitor and track the ingestion progress
4. Resource Management: Better control over CPU and network resource utilization

We use a conservative batch size of 50 to ensure reliable operation.
The optimal batch size depends on many factors including:
- Document sizes being inserted
- Available system resources
- Network conditions
- Concurrent workload

Consider measuring performance with your specific workload before adjusting.

In [20]:
batch_size = 50

# Automatic Batch Processing
articles = [article for article in unique_news_articles if article and len(article) <= 50000]

try:
    vector_store.add_texts(
        texts=articles,
        batch_size=batch_size
    )
    logging.info("Document ingestion completed successfully.")
except Exception as e:
    raise ValueError(f"Failed to save documents to vector store: {str(e)}")

2025-09-17 11:33:17 [INFO] Document ingestion completed successfully.


# Create the GSI Vector Index

Now that we have data in our vector store, we need to create a GSI vector index to enable high-performance vector searches. The index creation is done programmatically through the vector store, which will optimize the index settings based on our data and requirements.

We'll create a BHIVE index, which is optimized for pure vector searches and can scale to billions of vectors with excellent performance characteristics.

In [21]:
# Create GSI Vector Index for high-performance searches
try:
    # Create a BHIVE index optimized for pure vector searches
    vector_store.create_index(
        index_type=INDEX_TYPE,  # BHIVE index type
        index_name=INDEX_NAME,  # crew_gsi_index
        index_description=INDEX_DESCRIPTION  # IVF,SQ8 for optimized performance
    )
    print(f"GSI Vector index '{INDEX_NAME}' created successfully")
    logging.info(f"BHIVE index '{INDEX_NAME}' created with description '{INDEX_DESCRIPTION}'")
    
    # Wait a moment for index to be available
    print("Waiting for index to become available...")
    time.sleep(5)
    
except Exception as e:
    # Index might already exist, which is fine
    if "already exists" in str(e).lower():
        print(f"GSI Vector index '{INDEX_NAME}' already exists, proceeding...")
        logging.info(f"Index '{INDEX_NAME}' already exists")
    else:
        logging.error(f"Failed to create GSI index: {str(e)}")
        raise RuntimeError(f"GSI index creation failed: {str(e)}")


2025-09-17 11:33:22 [INFO] BHIVE index 'vector_search_crew' created with description 'IVF,SQ8'


GSI Vector index 'vector_search_crew' created successfully
Waiting for index to become available...


## Creating a Vector Search Tool
After loading our data into the vector store, we need to create a tool that can efficiently search through these vector embeddings. This involves two key components:

### Vector Retriever
The vector retriever is configured to perform similarity searches. This creates a retriever that performs semantic similarity searches against our vector database. The similarity search finds documents whose vector embeddings are closest to the query's embedding in the vector space.

### Search Tool
The search tool wraps the retriever in a user-friendly interface that:
- Takes a query string as input
- Passes the query to the retriever to find relevant documents
- Formats the results with clear document separation using document numbers and dividers
- Returns the formatted results as a single string with each document clearly delineated

The tool is designed to integrate seamlessly with our AI agents, providing them with reliable access to our knowledge base through vector similarity search. The lambda function in the tool handles both direct string queries and structured query objects, ensuring flexibility in how the tool can be invoked.


In [22]:
# Create GSI vector retriever optimized for high-performance searches
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # Return top 4 most similar documents
)

# Define the GSI vector search tool using the @tool decorator
@tool("gsi_vector_search")
def search_tool(query: str) -> str:
    """Search for relevant documents using GSI vector similarity.
    Input should be a simple text query string.
    Returns a list of relevant document contents from GSI vector search.
    Use this tool to find detailed information about topics using high-performance GSI indexes."""
    
    # Invoke the GSI vector retriever
    docs = retriever.invoke(query)

    # Format the results
    formatted_docs = "\n\n".join([
        f"Document {i+1}:\n{'-'*40}\n{doc.page_content}"
        for i, doc in enumerate(docs)
    ])
    return formatted_docs

# Creating CrewAI Agents

We'll create two specialized AI agents using the CrewAI framework to handle different aspects of our information retrieval and analysis system. This multi-agent approach allows us to:
- Leverage specialized expertise for different tasks
- Maintain high quality through separation of concerns
- Create more comprehensive and reliable outputs
- Scale the system's capabilities efficiently

In [23]:
# Custom response template
response_template = """
Analysis Results
===============
{%- if .Response %}
{{ .Response }}
{%- endif %}

Sources
=======
{%- for tool in .Tools %}
* {{ tool.name }}
{%- endfor %}

Metadata
========
* Confidence: {{ .Confidence }}
* Analysis Time: {{ .ExecutionTime }}
"""

The agents work together in a coordinated way:
1. Research agent finds and analyzes relevant documents
2. Writer agent takes those findings and crafts polished responses
3. Both agents use a custom response template for consistent output

## Research Expert Agent

This agent is designed to:
- Execute semantic searches using our vector store
- Analyze and evaluate search results 
- Identify key information and insights
- Verify facts across multiple sources
- Synthesize findings into comprehensive research summaries

In [24]:
# Create research agent
researcher = Agent(
    role='Research Expert',
    goal='Find and analyze the most relevant documents to answer user queries accurately',
    backstory="""You are an expert researcher with deep knowledge in information retrieval 
    and analysis. Your expertise lies in finding, evaluating, and synthesizing information 
    from various sources. You have a keen eye for detail and can identify key insights 
    from complex documents. You always verify information across multiple sources and 
    provide comprehensive, accurate analyses.""",
    tools=[search_tool],
    llm=llm,
    verbose=True,
    memory=True,
    allow_delegation=False,
    response_template=response_template
)

## Technical Writer Agent  

This agent is responsible for:
- Taking research findings and structuring them logically
- Converting technical concepts into clear explanations
- Ensuring proper citation and attribution
- Maintaining engaging yet informative tone
- Producing well-formatted final outputs

In [25]:
# Create writer agent
writer = Agent(
    role='Technical Writer',
    goal='Generate clear, accurate, and well-structured responses based on research findings',
    backstory="""You are a skilled technical writer with expertise in making complex 
    information accessible and engaging. You excel at organizing information logically, 
    explaining technical concepts clearly, and creating well-structured documents. You 
    ensure all information is properly cited, accurate, and presented in a user-friendly 
    manner. You have a talent for maintaining the reader's interest while conveying 
    detailed technical information.""",
    llm=llm,
    verbose=True,
    memory=True,
    allow_delegation=False,
    response_template=response_template
)

# How CrewAI Agents Work in this RAG System

### Agent-Based RAG Architecture

This system uses a two-agent approach to implement Retrieval-Augmented Generation (RAG):

1. **Research Expert Agent**:
   - Receives the user query
   - Uses the vector search tool to retrieve relevant documents from Couchbase
   - Analyzes and synthesizes information from retrieved documents
   - Produces a comprehensive research summary with key findings

2. **Technical Writer Agent**:
   - Takes the research summary as input
   - Structures and formats the information
   - Creates a polished, user-friendly response
   - Ensures proper attribution and citation

#### How the Process Works:

1. **Query Processing**: User query is passed to the Research Agent
2. **Vector Search**: Query is converted to embeddings and matched against document vectors
3. **Document Retrieval**: Most similar documents are retrieved from Couchbase
4. **Analysis**: Research Agent analyzes documents for relevance and extracts key information
5. **Synthesis**: Research Agent combines findings into a coherent summary
6. **Refinement**: Writer Agent restructures and enhances the content
7. **Response Generation**: Final polished response is returned to the user

This multi-agent approach separates concerns (research vs. writing) and leverages
specialized expertise for each task, resulting in higher quality responses.

# Testing the Search System

Test the system with some example queries.

In [26]:
def process_query(query, researcher, writer):
    """
    Test the complete RAG system with a user query.
    
    This function tests both the vector search capability and the agent-based processing:
    1. Vector search: Retrieves relevant documents from Couchbase
    2. Agent processing: Uses CrewAI agents to analyze and format the response
    
    The function measures performance and displays detailed outputs from each step.
    """
    print(f"\nQuery: {query}")
    print("-" * 80)
    
    # Create tasks
    research_task = Task(
        description=f"Research and analyze information relevant to: {query}",
        agent=researcher,
        expected_output="A detailed analysis with key findings and supporting evidence"
    )
    
    writing_task = Task(
        description="Create a comprehensive and well-structured response",
        agent=writer,
        expected_output="A clear, comprehensive response that answers the query",
        context=[research_task]
    )
    
    # Create and execute crew
    crew = Crew(
        agents=[researcher, writer],
        tasks=[research_task, writing_task],
        process=Process.sequential,
        verbose=True,
        cache=True,
        planning=True
    )
    
    try:
        start_time = time.time()
        result = crew.kickoff()
        elapsed_time = time.time() - start_time
        
        print(f"\nQuery completed in {elapsed_time:.2f} seconds")
        print("=" * 80)
        print("RESPONSE")
        print("=" * 80)
        print(result)
        
        if hasattr(result, 'tasks_output'):
            print("\n" + "=" * 80)
            print("DETAILED TASK OUTPUTS")
            print("=" * 80)
            for task_output in result.tasks_output:
                print(f"\nTask: {task_output.description[:100]}...")
                print("-" * 40)
                print(f"Output: {task_output.raw}")
                print("-" * 40)
    except Exception as e:
        print(f"Error executing crew: {str(e)}")
        logging.error(f"Crew execution failed: {str(e)}", exc_info=True)

In [27]:
# Disable logging before running the query
logging.disable(logging.CRITICAL)

query = "What are the key details about the FA Cup third round draw? Include information about Manchester United vs Arsenal, Tamworth vs Tottenham, and other notable fixtures."
process_query(query, researcher, writer)


Query: What are the key details about the FA Cup third round draw? Include information about Manchester United vs Arsenal, Tamworth vs Tottenham, and other notable fixtures.
--------------------------------------------------------------------------------


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 23b8a16a-4689-4ba4-9c07-63740605d035                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 
[2025-09-17 11:33:28][INFO]: Planning the crew execution
[EventBus Error] Handler 'on_task_started' failed for event 'TaskStartedEvent': 'NoneType' object has no attribute 'key'


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f7464ad8-7752-4411-af3b-fca6b5f0e19e                                                                     │
│  Agent: Task Execution Planner                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Expert                                                                                         │
│                                                                                                                 │
│  Task: Research and analyze information relevant to: What are the key details about the FA Cup third round      │
│  draw? Include information about Manchester United vs Arsenal, Tamworth vs Tottenham, and other notable         │
│  fixtures.1. Begin by clearly understanding the objectives of the research to ensure all key details regarding  │
│  the FA Cup third round draw are covered, especially focusing on the matches involving Manchester United vs     │
│  Arsenal and Tamworth vs Tottenham.                                                                             │
│  2. Utilize the 'gsi_vector_search' tool by constructing a specific query string that encapsulates the task.    │
│  For example, the query could be: 'key details about the FA Cup third round draw, including Manchester United   │
│  vs Arsenal and Tamworth vs Tottenham.'                                                                         │
│  3. Execute the search with the GSI vector similarity tool, ensuring to verify the results provided. Aim to     │
│  obtain comprehensive and pertinent documents that highlight the match schedules, team form, and historical     │
│  performance in the FA Cup.                                                                                     │
│  4. Analyze the documents retrieved, taking notes on critical findings related to both matchups, any notable    │
│  fixtures, and context such as current team standings or player injuries that might impact the games.           │
│  5. Synthesize the findings into a cohesive narrative that outlines the essential details needed in the         │
│  analysis.                                                                                                      │
│  6. Prepare to summarize the key information and prepare data to support each finding clearly.                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Writer                                                                                        │
│                                                                                                                 │
│  Task: Create a comprehensive and well-structured response1. Receive the comprehensive analysis results from    │
│  Task Number 1, ensuring to gather all the detailed insights documented by the Research Expert.                 │
│  2. Review the findings to identify the main themes and key points that need to be included in the response.    │
│  Focus particularly on the specific matches of Manchester United vs Arsenal and Tamworth vs Tottenham.          │
│  3. Begin drafting the response by outlining sections that cover an introduction, a detailed analysis of the    │
│  key matches with relevant statistics, and any additional notable fixtures.                                     │
│  4. Write the introduction highlighting the importance of the FA Cup third round and what fans can look         │
│  forward to with the outlined matches.                                                                          │
│  5. In the main body, present a structured analysis, including each matchup separately, detailing team          │
│  performance metrics, player focus, and any other relevant insights from the analysis conducted. Utilize        │
│  bullet points or numbered lists to enhance readability.                                                        │
│  6. Conclude the response by summarizing the significance of the upcoming matches, potentially predicting       │
│  outcomes and encouraging fan engagement.                                                                       │
│  7. Revise the entire response for clarity, coherence, and completeness. Make sure formatting is appropriate    │
│  and the document is free from errors before finalizing.                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Writer                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Introduction**                                                                                               │
│                                                                                                                 │
│  The FA Cup third round is a pivotal moment in the English football calendar, marking the entry of Premier      │
│  League and Championship clubs into the competition. This stage is renowned for its potential for thrilling     │
│  matchups and unexpected outcomes, often referred to as "giant-killings." Fans eagerly anticipate the drama     │
│  and excitement that the FA Cup consistently delivers. This year's draw has set the stage for some captivating  │
│  fixtures, with Manchester United facing Arsenal and Tamworth hosting Tottenham Hotspur being the standout      │
│  matches. These games promise to deliver high-stakes football and showcase the diverse talent across English    │
│  football's tiers.                                                                                              │
│                                                                                                                 │
│  **Detailed Analysis of Key Matches**                                                                           │
│                                                                                                                 │
│  1. **Manchester United vs Arsenal**                                                                            │
│     - **Historical Context**: This fixture is a clash between two of the most successful clubs in FA Cup        │
│  history. Manchester United, the current holders, face Arsenal, who have won the competition a record 14        │
│  times. The match is scheduled for the weekend of Saturday, 11 January.                                         │
│     - **Team Performance Metrics**:                                                                             │
│       - *Manchester United*: The team is currently navigating internal dynamics, particularly with star player  │
│  Marcus Rashford, who has expressed interest in exploring new opportunities. Manager Ruben Amorim is focusing   │
│  on maintaining team cohesion and leveraging Rashford's skills to enhance United's FA Cup campaign.             │
│       - *Arsenal*: Known for their strong FA Cup pedigree, Arsenal will aim to capitalize on their historical   │
│  success and current form to advance further in the competition.                                                │
│     - **Player Focus**: Marcus Rashford's performance will be crucial for Manchester United, while Arsenal      │
│  will rely on their experienced players to guide them through this challenging fixture.                         │
│                                                                                                                 │
│  2. **Tamworth vs Tottenham Hotspur**                                                                           │
│     - **Significance**: Tamworth, one of only two non-league clubs remaining, has reached this stage after a    │
│  remarkable penalty shootout victory against League One's Burton Albion. Hosting a Premier League giant like    │
│  Tottenham is a significant achievement for the club, which is the lowest-ranked team left in the competition.  │
│     - **Team Performance Metrics**:                    

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: adcb03a5-1fdf-4d4e-be6f-d64df6952960                                                                     │
│  Agent: Technical Writer                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 23b8a16a-4689-4ba4-9c07-63740605d035                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Introduction**                                                                                 │
│                                                                                                                 │
│  The FA Cup third round is a pivotal moment in the English football calendar, marking the entry of Premier      │
│  League and Championship clubs into the competition. This stage is renowned for its potential for thrilling     │
│  matchups and unexpected outcomes, often referred to as "giant-killings." Fans eagerly anticipate the drama     │
│  and excitement that the FA Cup consistently delivers. This year's draw has set the stage for some captivating  │
│  fixtures, with Manchester United facing Arsenal and Tamworth hosting Tottenham Hotspur being the standout      │
│  matches. These games promise to deliver high-stakes football and showcase the diverse talent across English    │
│  football's tiers.                                                                                              │
│                                                                                                                 │
│  **Detailed Analysis of Key Matches**                                                                           │
│                                                                                                                 │
│  1. **Manchester United vs Arsenal**                                                                            │
│     - **Historical Context**: This fixture is a clash between two of the most successful clubs in FA Cup        │
│  history. Manchester United, the current holders, face Arsenal, who have won the competition a record 14        │
│  times. The match is scheduled for the weekend of Saturday, 11 January.                                         │
│     - **Team Performance Metrics**:                                                                             │
│       - *Manchester United*: The team is currently navigating internal dynamics, particularly with star player  │
│  Marcus Rashford, who has expressed interest in exploring new opportunities. Manager Ruben Amorim is focusing   │
│  on maintaining team cohesion and leveraging Rashford's skills to enhance United's FA Cup campaign.             │
│       - *Arsenal*: Known for their strong FA Cup pedigree, Arsenal will aim to capitalize on their historical   │
│  success and current form to advance further in the competition.                                                │
│     - **Player Focus**: Marcus Rashford's performance will be crucial for Manchester United, while Arsenal      │
│  will rely on their experienced players to guide them through this challenging fixture.                         │
│                                                                                                                 │
│  2. **Tamworth vs Tottenham Hotspur**                                                                           │
│     - **Significance**: Tamworth, one of only two non-league clubs remaining, has reached this stage after a    │
│  remarkable penalty shootout victory against League One's Burton Albion. Hosting a Premier League giant like    │
│  Tottenham is a significant achievement for the club, 


Query completed in 46.00 seconds
RESPONSE
**Introduction**

The FA Cup third round is a pivotal moment in the English football calendar, marking the entry of Premier League and Championship clubs into the competition. This stage is renowned for its potential for thrilling matchups and unexpected outcomes, often referred to as "giant-killings." Fans eagerly anticipate the drama and excitement that the FA Cup consistently delivers. This year's draw has set the stage for some captivating fixtures, with Manchester United facing Arsenal and Tamworth hosting Tottenham Hotspur being the standout matches. These games promise to deliver high-stakes football and showcase the diverse talent across English football's tiers.

**Detailed Analysis of Key Matches**

1. **Manchester United vs Arsenal**
   - **Historical Context**: This fixture is a clash between two of the most successful clubs in FA Cup history. Manchester United, the current holders, face Arsenal, who have won the competition a reco

# Conclusion

By following these steps, you've built a powerful RAG system that combines Couchbase's high-performance GSI vector storage capabilities with CrewAI's agent-based architecture. This multi-agent approach leverages the advanced features of GSI vector search to deliver superior performance and scalability.

The system demonstrates several key advantages:
1. Efficient vector search using Couchbase's Query vector store
2. Specialized AI agents that focus on different aspects of the RAG pipeline
3. Collaborative workflow between agents to produce comprehensive, well-structured responses
4. Scalable architecture that can be extended with additional agents for more complex tasks

Whether you're building a customer support system, a research assistant, or a knowledge management solution, this agent-based RAG approach provides a flexible foundation that can be adapted to various use cases and domains.